In [1]:
import pandas as pd
import numpy as np
from Bio import AlignIO, SeqIO
from Bio.Seq import Seq
from Bio.SeqRecord import SeqRecord
from collections import Counter
from Levenshtein import distance
from copy import deepcopy
from joblib import Parallel, delayed

In [2]:
protease_consensus = SeqIO.read('sequences/hiv-protease-consensus.fasta', 'fasta')
protease_sequences = [s for s in SeqIO.parse('sequences/HIV-1_pol-protease.fasta', 'fasta') if len(s.seq) >= len(protease_consensus.seq)]
rt_consensus = SeqIO.read('sequences/hiv-rt-consensus.fasta', 'fasta')
rt_sequences = [s for s in SeqIO.parse('sequences/HIV-1_pol-RT.fasta', 'fasta') if len(s.seq) >= len(rt_consensus.seq)]

In [3]:
len(rt_consensus)

560

In [4]:
Counter([len(s) for s in rt_sequences])

Counter({560: 624,
         561: 26,
         562: 224,
         563: 1,
         564: 2,
         565: 1,
         566: 5,
         567: 1,
         568: 1,
         569: 5,
         570: 10,
         571: 16,
         573: 5,
         574: 1,
         575: 1,
         578: 4,
         579: 1,
         580: 2,
         581: 2,
         582: 2,
         583: 7,
         584: 20,
         585: 6,
         586: 5,
         587: 5,
         588: 8,
         589: 8,
         590: 8,
         591: 12,
         592: 20,
         593: 15,
         594: 47,
         595: 54,
         596: 24,
         597: 21,
         598: 26,
         599: 43,
         600: 9,
         601: 13,
         602: 8,
         603: 2,
         604: 9,
         605: 6,
         606: 8,
         607: 7,
         608: 3,
         609: 4,
         610: 1,
         611: 2,
         612: 3,
         613: 2,
         615: 5,
         616: 1,
         617: 2,
         618: 4,
         619: 4,
         620: 2,
         621:

In [ ]:
# Define a function to extract out the protease sequence, even if it's not equal to consensus.

def find_best_match(consensus, query):
    """
    Pass in two SeqRecords. The conversion to strings will be done inside the function.
    """
    query_str = str(query.seq) # <- this is greater than 99 a.a. long.

    min_dist = np.inf
    best_pos = 0
    for pos in range(len(query_str)):
        d_current = distance(str(consensus.seq), str(query_str[pos:pos + len(consensus.seq)]))
        if d_current < min_dist:
            min_dist = d_current
            best_pos = pos
          
    matching_query = SeqRecord(seq=query.seq[best_pos:best_pos+len(consensus.seq)], id=query.id, description=query.description, name=query.name)
    return matching_query

In [ ]:
cleaned_protease_sequences = Parallel(n_jobs=-1)(delayed(find_best_match)(protease_consensus, s) for s in protease_sequences)
len(cleaned_protease_sequences)

In [ ]:
cleaned_rt_sequences = Parallel(n_jobs=-1)(delayed(find_best_match)(rt_consensus, s) for s in rt_sequences)
len(cleaned_rt_sequences)

In [ ]:
SeqIO.write(cleaned_protease_sequences, 'sequences/HIV1-protease.fasta', 'fasta')
SeqIO.write(cleaned_rt_sequences, 'sequences/HIV1-RT.fasta', 'fasta')

In [ ]:
protease_sequences[101236].seq[26:26+99]